In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras import layers, preprocessing
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pyarrow.feather as feather
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout, BatchNormalization,LSTM
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
edf_path="/content/drive/MyDrive/temp2/signal_sync_EDF ("

In [ ]:
edf = pd.DataFrame()

for i in range (2,24):
  path = edf_path + str(i) + ').feather'
  edf_temp = feather.read_table(path).to_pandas()
  edf_temp['Patient'] = i
  edf = pd.concat([edf, edf_temp.iloc[::100,:]])

In [ ]:
y = np.array([ 49.21925595452256, 79.17007141080755, 31.33331048898331, 36.30885122410546, 70.04614242844697,
              51.50981226076919, 24.066003807911994, 24.066003807911994, 44.145334434351774, 23.339089339271105,
               56.37456501107244, 56.37456501107244, 39.90900644003716, 29.995583689091713, 33.02068965517241,
               51.50981226076919, 22.309859154929576, 43.288490284005974, 61.88520676153185, 18.013110808499857,
               40.34420652391435, 32.98709474346868
               ])

In [ ]:
max_length = edf.groupby('Patient')['SpO2'].count().max()
X = [np.pad(edf[edf['Patient'] == patient]['SpO2'].values, (0, max_length - len(edf[edf['Patient'] == patient]))) for patient in edf['Patient'].unique()]
X = np.array(X)

In [ ]:
X = np.expand_dims(X, axis=2)
y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
keras.utils.set_random_seed(42)

In [ ]:
model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(max_length, 1)),
    BatchNormalization(),
    Conv1D(64, 3, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Flatten(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=6, validation_split=0.2)

In [ ]:
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test MAE: {mae}")

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
import pickle

# Save the model to disk
filename = 'AHI_model_ML_0909.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model, file)
